In [ ]:
#! pip install pandas
#! pip install numpy 
#! pip install scikit-learn


In [1]:
import pandas as pd
import numpy as np


In [2]:
#import dataset
data = pd.read_csv('language_dataset.csv')

In [4]:
#check for the shape of dataset
data.shape

(22000, 3)

In [ ]:
#check for missing values
data.isnull().sum()

Text        0
language    0
dtype: int64

In [ ]:
#drop unnecessary column
data.drop(columns='Unnamed: 0', inplace=True)
data.columns

In [69]:
#check for number of observations per language
data['language'].value_counts()

language
Estonian      1000
Swedish       1000
English       1000
Russian       1000
Romanian      1000
Persian       1000
Pushto        1000
Spanish       1000
Hindi         1000
Korean        1000
Chinese       1000
French        1000
Portugese     1000
Indonesian    1000
Urdu          1000
Latin         1000
Turkish       1000
Japanese      1000
Dutch         1000
Tamil         1000
Thai          1000
Arabic        1000
Name: count, dtype: int64

In [70]:
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split


In [71]:
x_data = data[['Text']]
y_data = data[['language']]

In [81]:
x_train, x_test, y_train, y_test = train_test_split(x_data, y_data, test_size= 0.3, random_state=34, stratify = y_data)

In [82]:
y_train.value_counts()

language  
Arabic        700
Chinese       700
Turkish       700
Thai          700
Tamil         700
Swedish       700
Spanish       700
Russian       700
Romanian      700
Pushto        700
Portugese     700
Persian       700
Latin         700
Korean        700
Japanese      700
Indonesian    700
Hindi         700
French        700
Estonian      700
English       700
Dutch         700
Urdu          700
Name: count, dtype: int64

In [83]:
y_test.value_counts()

language  
Arabic        300
Chinese       300
Turkish       300
Thai          300
Tamil         300
Swedish       300
Spanish       300
Russian       300
Romanian      300
Pushto        300
Portugese     300
Persian       300
Latin         300
Korean        300
Japanese      300
Indonesian    300
Hindi         300
French        300
Estonian      300
English       300
Dutch         300
Urdu          300
Name: count, dtype: int64